In [294]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import geopandas as gpd

In [295]:
# load distance matrix, orig w/ additional distance, pal data (qty, geodata)
distM = pd.read_csv(r'C:\Users\isaldiviagonzatti\Downloads\gitFiles\MScThesisCode\FLP\output\distMatrix.csv', index_col=0)
orig =  pd.read_csv(r'C:\Users\isaldiviagonzatti\Downloads\gitFiles\MScThesisCode\FLP\output\nearestnodePAL.csv',  header=None).T
pal = r'C:\Users\isaldiviagonzatti\Downloads/gitFiles/MScThesisCode/FLP/output/ northPAL.gpkg'

In [296]:
# rename and transform orig data
orig = orig.rename(columns={0: 'osmidOrig', 1: 'distance'})
orig.osmidOrig = orig.osmidOrig.astype(np.int64)  

In [297]:
# read gdp file
palPoint = gpd.read_file(pal) 
# define lon, lat points and osmid from orig 
# Note that here we assume (and check) that osmid column follows same order as provided when
# calculating nearest node. There must be a way of getting the geodata for which osmid was generated
# but we did not look into it
palPoint['xlon']=palPoint['geometry'].x
palPoint['ylat']=palPoint['geometry'].y
palPoint[['osmidOrig', 'fieldDist']] = orig[['osmidOrig', 'distance']]

In [298]:
# aggregate PAL data based on nodes 
palAgg = pd.DataFrame(palPoint.groupby('osmidOrig', as_index=False).agg(
                                            osmidOrig=('osmidOrig','mean'), 
                                            areaHa= ('areaHa','sum'),
                                            leavesTonne=('leavesTonne','sum'),
                                            distance = ('fieldDist', 'sum')
                                        )
                     )

palAgg.osmidOrig = palAgg.osmidOrig.astype(np.int64)  

In [299]:
# merge distances with PAL data
optiData = distM.merge(palAgg, left_on='origs', right_on='osmidOrig')

In [300]:
# calculate total distance (road + field centroids to node) and cost per trip
optiData['totalDist'] = (optiData.path_length + optiData.distance)/1000
optiData['costTrans'] = optiData.totalDist  * 0.20 # 0.20 = transport cost tkm

In [301]:
#save data and look at result
optiData.to_csv(r'C:\Users\isaldiviagonzatti\Downloads\gitFiles\MScThesisCode\FLP\output\optiData.csv')
optiData.head()

,origs,dests,path_length,osmidOrig,areaHa,leavesTonne,distance,totalDist,costTrans
0,3610149725,2656380182,157664.026,3610149725,15.49063,2540.463291,319.129599,157.983156,31.596631
1,3610149725,5294901777,83956.035,3610149725,15.49063,2540.463291,319.129599,84.275165,16.855033
2,3610149725,1321571077,201954.500,3610149725,15.49063,2540.463291,319.129599,202.273630,40.454726
3,3610149725,3677820110,75493.804,3610149725,15.49063,2540.463291,319.129599,75.812934,15.162587
4,3610149725,3510580697,21814.095,3610149725,15.49063,2540.463291,319.129599,22.133225,4.426645
